This is an example for the `regpy.solvers.nonlinear.irgnm.IrgnmCG` solver for the example of __X-ray phase retriavel__.

In X-ray phase retrieval, we assume that a coherent plain wave hits a shallow monomaterial object that absorbs part of the beam intensity and phase shifts the rest, depending on the density of material. The ratio of shifting and abortion is given by $\beta/\delta$. The wave the propagates behind the object and then the intensity of the wave (hologram) is measured at some point.  The phase information is lost and needs to be recovered.

The forward operator $F$ of X-ray phase contrast imaging is defined by

$$
    F(\phi) = |D_F(exp(-(i + c_{\beta/\delta}) \cdot  phi))|^2 = I
$$

where $D_F$ is the Fresnel-propagator and $c_{\beta/\delta}$ is
a constant that parametrizes the magnitude of X-ray absorption versus X-ray
refraction for the imaged sample ( $c_{\beta/\delta} = \beta/\delta$).

![image_phase_retrival.png](image_phase_retrival.png)
The image is from the [phd thesis of Simon Maretzke (2019)](https://ediss.uni-goettingen.de/handle/21.11130/00-1735-0000-0003-C12B-3)

The __Iteratively Regularized Gauss-Newton Method method__ minimizes in each iteration: 
$$
    \Vert(x_{n}) + T'[x_n] h - data\Vert^{2} + regpar_{n} \cdot \Vert x_{n} + h - init\Vert^{2}
$$
where $T$ is a Frechet-differentiable operator, using `regpy.solvers.linear.tikhonov.TikhonovCG`.
$regpar_n$ is a decreasing geometric sequence of regularization parameters.

In [ ]:
from regpy.solvers.nonlinear.irgnm import IrgnmCG

from xray_phase_contrast_operator import get_xray_phase_contrast
from regpy.hilbert import L2
from regpy.vecsps import UniformGridFcts
from regpy.solvers import RegularizationSetting
import regpy.stoprules as rules

import numpy as np
from scipy.datasets import ascent
import logging
import matplotlib.pyplot as plt

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-20s :: %(message)s'
)


We first have to define the forward propagation operator for that we have to define a grid and a fresnel number, which is encoding the propagation distance

In [ ]:


# Example parameters
fresnel_number = 5e-4    # Fresnel-number of the simulated imaging system, associated with the unit-lengthscale
                         # in grid (i.e. with the size of one pixel for the above choice of grid)
noise_level = 0.01      # Noise level in the simulated data



In [ ]:

# Uniform grid of unit-spacing
grid = UniformGridFcts(np.arange(1024), np.arange(1024))

# Forward operator
op = get_xray_phase_contrast(grid, fresnel_number)


We build our simulated data using the ascent image in the scipy library

In [ ]:

# Create phantom phase-image (= padded example-image)
exact_solution = ascent().astype(np.float64)
#normalizing the image
exact_solution /= exact_solution.max()
#The wave needs to propagated and for that some room around the signal is necessary there for we add padding
pad_amount = tuple([(grid.shape[0] - exact_solution.shape[0])//2,
                    (grid.shape[1] - exact_solution.shape[1])//2])
exact_solution = np.pad(exact_solution, pad_amount, 'constant', constant_values=0)



fig,axs = plt.subplots(1,2,figsize=(12,5))
ax = axs[0]
im = ax.imshow(ascent().astype(np.float64))
ax.set_title("scipy ascent image")
plt.colorbar(im,ax=ax)
ax = axs[1]
im = plt.imshow(exact_solution)
ax.set_title("scaled and paded")
plt.colorbar(im,ax=ax)


In [ ]:

# Create exact and noisy data
exact_data = op(exact_solution)
noise = noise_level * op.codomain.randn()
data = exact_data + noise
fig,axs = plt.subplots(1,2,figsize=(12,5))
ax = axs[0]
im = ax.imshow(exact_data)
ax.set_title("exact data")
plt.colorbar(im,ax=ax)
ax = axs[1]
im = plt.imshow(data)
ax.set_title(f"datawith {noise_level} noise")
plt.colorbar(im,ax=ax)


We then can __set up__ the __solver__ `regpy.solvers.nonlinear.irgnm.IrgnmCG` by building a __setting__ using the operator and the `L2` spaces, which we then give into the solver together with the __`data`__ and a regularization parameter __`regpar`__ 

In [ ]:

# Image-reconstruction using the IRGNM method
#define setting with operator and its domain and co-domain
setting = RegularizationSetting(op=op, penalty=L2, data_fid=L2)
#Setup the solver
solver = IrgnmCG(setting, data, regpar=10)


We also need to define a stopping rule here we combine a maximum amount of iterations and the discrepancy principle (for more see `regpy.stoprules.rules`).  

In [ ]:

#define the stoprule from maximum amount of iterations and discrepancy principle
stoprule = (
    rules.CountIterations(max_iterations=30) +
    rules.Discrepancy(
        setting.h_codomain.norm,
        data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=1.1
    )
)


We now can run the solver. 

In [ ]:
#run the solver with stopping rule
reco, reco_data = solver.run(stoprule)

In [ ]:

# Plot results
fig,axs = plt.subplots(1,3,figsize = (20,5))
ax = axs[0]
ax.set_title('Simulated data (hologram)')
im = ax.imshow(data)
fig.colorbar(im,ax=ax)

ax = axs[1]
ax.set_title('Reconstruction (phase-image)')
im = ax.imshow(reco)
fig.colorbar(im,ax=ax)

ax = axs[2]
ax.set_title('Exact solution (phase-image)')
im = ax.imshow(exact_solution)
fig.colorbar(im,ax = ax)

plt.show()
